# Feature Selection for log_diff_model (Multivariate Model)

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from bdi_predict.ml_logic.sequence_gen import WindowGenerator
from tensorflow.keras import layers
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
bdi = pd.read_csv("../raw_data/data/BDI/log_diff_BDI_daily.csv")
bdi.head(2)

,Unnamed: 0,Date,Price,abs_price,log_price,log_diff
0,0,1995-01-03,1964.0,NaN,3.293141,NaN
1,1,1995-01-04,1961.0,-3.0,3.292478,-0.000664


In [3]:
bdi["Date"] = pd.to_datetime(bdi["Date"])
bdi.set_index("Date", inplace=True)
bdi= bdi[bdi.index != "1995-01-03"]
bdi.drop(columns="Unnamed: 0", inplace=True)
bdi.head(2)

,Price,abs_price,log_price,log_diff
Date,,,,
1995-01-04,1961.0,-3.0,3.292478,-0.000664
1995-01-05,1967.0,6.0,3.293804,0.001327


In [4]:
merged = pd.read_csv("../raw_data/data/merged_daily_data.csv")
merged["Date"] = pd.to_datetime(merged["Date"])
merged.set_index("Date", inplace=True)
cip = merged[["CIP"]]

In [5]:
feat = pd.read_csv("../raw_data/data/merged_features.csv")
feat["Date"] = pd.to_datetime(feat["Date"])
feat.set_index("Date", inplace=True)
feat.drop(columns="Unnamed: 0", inplace=True)
feat = feat[["close_y", "Nickel_x", "DAP"]]
feat.rename(columns={"close_y":"iron", 
                    "Nickel_x":"nickel", 
                    "DAP":"DAP"}, inplace=True)
comb=feat
comb

,iron,nickel,DAP
Date,,,
1995-01-03,12.270000,8505.450000,198.630000
1995-01-04,12.270000,8505.450000,198.630000
1995-01-05,12.270000,8505.450000,198.630000
1995-01-06,12.270000,8505.450000,198.630000
1995-01-09,12.270000,8505.450000,198.630000
...,...,...,...
2022-11-08,27.540690,10264.567966,306.576271
2022-11-09,26.013621,10003.955424,297.296610
2022-11-10,24.486552,9743.342881,288.016949


In [6]:
cass = pd.read_csv("../raw_data/data/CASS/CASS FREIGHT INDEX.csv")
cass = cass[["time", "close"]]
cass["time"] = pd.to_datetime(cass["time"], origin="unix")
cass.set_index("time", inplace=True)
datetime_index = pd.date_range(start="1994-01-01", end="2021-12-01", name="Date")
len(datetime_index)
cass_df = pd.DataFrame(index=datetime_index)
cass_df["cass"] = cass["close"]
cass_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10197 entries, 1994-01-01 to 2021-12-01
Freq: D
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cass    0 non-null      float64
dtypes: float64(1)
memory usage: 159.3 KB


In [7]:
df = pd.merge(bdi, cip, on="Date", how="inner")
#df.drop(columns="Price_y", inplace=True)


In [8]:
df.head(3)

,Price,abs_price,log_price,log_diff,CIP
Date,,,,,
1995-01-04,1961.0,-3.0,3.292478,-0.000664,11.988118
1995-01-05,1967.0,6.0,3.293804,0.001327,12.173925
1995-01-06,1983.0,16.0,3.297323,0.003518,12.354720


# Data Cleaning

## Holdout Method (manual, chronological)

In [9]:
input_length = len(df)

In [10]:
def train_test_split(df:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> tuple:
    '''
    Returns a train dataframe and a test dataframe (df_train, df_test)
    from which one can sample (X,y) sequences using TimeseriesGenerator.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    
    # TRAIN SET

    last_train_idx = round(train_test_ratio * len(df))
    df_train = df.iloc[0:last_train_idx, :]

    # TEST SET
 
    first_test_idx = last_train_idx - input_length
    df_test = df.iloc[first_test_idx:, :]

    return (df_train, df_test)

In [11]:
df_train, df_test = train_test_split(df=df, train_test_ratio=0.8, input_length=input_length)

In [12]:
X_train = df_train[["Price", "log_price", "CIP"]]
y_train = df_train["log_diff"]

X_test  = df_test[["Price", "log_price", "CIP"]]
y_test = df_test["log_diff"]

In [13]:
#Feature Scaling

scaler_X = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.fit_transform(X_test)



In [20]:
generator = TimeseriesGenerator(X_train_scaled, y_train, length=20, batch_size=8, sampling_rate=1, stride=1)

In [15]:
len(generator)

704

In [16]:
generator_val = TimeseriesGenerator(X_test_scaled, y_test, length=20, batch_size=8, sampling_rate=1, stride=1)

In [22]:
for X, y in generator:
    print(X.shape, y.shape)
    print(X[0], y[0])
    #listy = [float(X[0][i]) for i in range(len(X[0]))]
    #print(listy)
    break

(8, 20, 3) (8,)
[[0.14519214 0.51537505 0.39141632]
 [0.14571379 0.51620029 0.39855159]
 [0.14710485 0.51838869 0.40549438]
 [0.14806121 0.51988299 0.42519177]
 [0.14988698 0.522713   0.43138855]
 [0.15023474 0.52324871 0.43740479]
 [0.15188663 0.52577889 0.4432429 ]
 [0.15379934 0.52867928 0.44890526]
 [0.15440793 0.52959564 0.46486177]
 [0.15475569 0.53011788 0.46984505]
 [0.15475569 0.53011788 0.47466453]
 [0.15466875 0.52998741 0.47932262]
 [0.1537124  0.52854812 0.48382169]
 [0.15240828 0.526573   0.49638873]
 [0.15136498 0.52498245 0.50027566]
 [0.1501478  0.52311488 0.50401553]
 [0.14823509 0.5201538  0.50761073]
 [0.14710485 0.51838869 0.51106366]
 [0.1465832  0.51757012 0.52059267]
 [0.14623544 0.51702302 0.52350039]] -0.0015435717560743


# The Model

In [20]:
# Learning Rate Scheduler: 

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)

In [21]:
#Optimizers

rmsprop = RMSprop(learning_rate=lr_schedule) #or 0.001 for inital test
adam = Adam(learning_rate=lr_schedule)

In [22]:
#Early Stopping Criterion:

es = EarlyStopping(monitor="val_loss", patience=500, restore_best_weights=True)

In [28]:
#Initializing and compiling model:

def init_model():
    
    
    model = Sequential()

    #LSTM layers
    model.add(layers.LSTM(60, activation="tanh", input_shape=(20,3), return_sequences=False))

    #Dense layers
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(1, activation="linear"))

    #compiling model
    model.compile(loss="mse", optimizer=rmsprop, metrics="mae")
    
    return model

# Training and Testing

In [29]:
#Instantiating model

model = init_model()

In [30]:
#Model layers & params overview:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 60)                15360     
                                                                 
 dense_2 (Dense)             (None, 100)               6100      
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21,561
Trainable params: 21,561
Non-trainable params: 0
_________________________________________________________________


In [31]:
#TRAINING THE MODEL:

history = model.fit(
    generator, epochs=1000, validation_data=generator_val, shuffle=True, callbacks=es
)

Epoch 1/1000
704/704 [==============================] - 4s 4ms/step - loss: 2.1441e-04 - mae: 0.0097 - val_loss: 1.4555e-04 - val_mae: 0.0096
Epoch 2/1000
704/704 [==============================] - 3s 4ms/step - loss: 7.3107e-05 - mae: 0.0060 - val_loss: 1.4613e-04 - val_mae: 0.0097
Epoch 3/1000
704/704 [==============================] - 3s 4ms/step - loss: 6.2392e-05 - mae: 0.0054 - val_loss: 1.0190e-04 - val_mae: 0.0077
Epoch 4/1000
704/704 [==============================] - 3s 4ms/step - loss: 6.0133e-05 - mae: 0.0053 - val_loss: 1.0118e-04 - val_mae: 0.0076
Epoch 5/1000
704/704 [==============================] - 3s 4ms/step - loss: 6.0049e-05 - mae: 0.0054 - val_loss: 1.0209e-04 - val_mae: 0.0077
Epoch 6/1000
704/704 [==============================] - 3s 4ms/step - loss: 5.9753e-05 - mae: 0.0054 - val_loss: 1.0407e-04 - val_mae: 0.0077
Epoch 7/1000
704/704 [==============================] - 3s 4ms/step - loss: 5.9188e-05 - mae: 0.0053 - val_loss: 1.0633e-04 - val_mae: 0.0077
Epoch 

704/704 [==============================] - 3s 4ms/step - loss: 1.8427e-05 - mae: 0.0027 - val_loss: 3.5967e-05 - val_mae: 0.0038
Epoch 116/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8393e-05 - mae: 0.0027 - val_loss: 3.6285e-05 - val_mae: 0.0037
Epoch 117/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8306e-05 - mae: 0.0027 - val_loss: 3.8031e-05 - val_mae: 0.0041
Epoch 118/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8174e-05 - mae: 0.0027 - val_loss: 3.8499e-05 - val_mae: 0.0040
Epoch 119/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8163e-05 - mae: 0.0027 - val_loss: 4.0808e-05 - val_mae: 0.0040
Epoch 120/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8331e-05 - mae: 0.0027 - val_loss: 3.6059e-05 - val_mae: 0.0038
Epoch 121/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.8390e-05 - mae: 0.0027 - val_loss: 3.4862e-05 - val_mae: 0.0035
Epoch 1

Epoch 172/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7236e-05 - mae: 0.0026 - val_loss: 4.8767e-05 - val_mae: 0.0046
Epoch 173/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7243e-05 - mae: 0.0026 - val_loss: 3.8368e-05 - val_mae: 0.0040
Epoch 174/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7060e-05 - mae: 0.0026 - val_loss: 3.9281e-05 - val_mae: 0.0039
Epoch 175/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7173e-05 - mae: 0.0026 - val_loss: 4.1269e-05 - val_mae: 0.0041
Epoch 176/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7067e-05 - mae: 0.0026 - val_loss: 4.2213e-05 - val_mae: 0.0041
Epoch 177/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7383e-05 - mae: 0.0026 - val_loss: 4.0982e-05 - val_mae: 0.0042
Epoch 178/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.7386e-05 - mae: 0.0026 - val_loss: 4.1228e-05 - val_mae:

Epoch 229/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6443e-05 - mae: 0.0025 - val_loss: 3.9900e-05 - val_mae: 0.0041
Epoch 230/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6553e-05 - mae: 0.0025 - val_loss: 3.9732e-05 - val_mae: 0.0041
Epoch 231/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6598e-05 - mae: 0.0025 - val_loss: 4.3392e-05 - val_mae: 0.0042
Epoch 232/1000
704/704 [==============================] - 3s 5ms/step - loss: 1.6804e-05 - mae: 0.0025 - val_loss: 4.0888e-05 - val_mae: 0.0042
Epoch 233/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6659e-05 - mae: 0.0025 - val_loss: 4.2646e-05 - val_mae: 0.0045
Epoch 234/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6482e-05 - mae: 0.0025 - val_loss: 4.3986e-05 - val_mae: 0.0043
Epoch 235/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.6638e-05 - mae: 0.0025 - val_loss: 4.0173e-05 - val_mae:

Epoch 286/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5935e-05 - mae: 0.0024 - val_loss: 4.0947e-05 - val_mae: 0.0043
Epoch 287/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5722e-05 - mae: 0.0024 - val_loss: 3.8597e-05 - val_mae: 0.0041
Epoch 288/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5821e-05 - mae: 0.0024 - val_loss: 3.8937e-05 - val_mae: 0.0041
Epoch 289/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5953e-05 - mae: 0.0024 - val_loss: 4.0907e-05 - val_mae: 0.0043
Epoch 290/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5732e-05 - mae: 0.0024 - val_loss: 4.0732e-05 - val_mae: 0.0042
Epoch 291/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5950e-05 - mae: 0.0024 - val_loss: 3.8677e-05 - val_mae: 0.0042
Epoch 292/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5638e-05 - mae: 0.0024 - val_loss: 4.3218e-05 - val_mae:

Epoch 343/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5422e-05 - mae: 0.0024 - val_loss: 4.6206e-05 - val_mae: 0.0046
Epoch 344/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5348e-05 - mae: 0.0024 - val_loss: 4.2681e-05 - val_mae: 0.0043
Epoch 345/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5431e-05 - mae: 0.0024 - val_loss: 4.4315e-05 - val_mae: 0.0046
Epoch 346/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5264e-05 - mae: 0.0024 - val_loss: 4.3510e-05 - val_mae: 0.0042
Epoch 347/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5465e-05 - mae: 0.0024 - val_loss: 4.2678e-05 - val_mae: 0.0043
Epoch 348/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5282e-05 - mae: 0.0024 - val_loss: 4.4871e-05 - val_mae: 0.0044
Epoch 349/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5480e-05 - mae: 0.0024 - val_loss: 4.2785e-05 - val_mae:

Epoch 400/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5108e-05 - mae: 0.0024 - val_loss: 4.6583e-05 - val_mae: 0.0044
Epoch 401/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4995e-05 - mae: 0.0024 - val_loss: 4.6273e-05 - val_mae: 0.0045
Epoch 402/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5014e-05 - mae: 0.0024 - val_loss: 4.7050e-05 - val_mae: 0.0044
Epoch 403/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4940e-05 - mae: 0.0024 - val_loss: 4.6617e-05 - val_mae: 0.0045
Epoch 404/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4880e-05 - mae: 0.0023 - val_loss: 4.7263e-05 - val_mae: 0.0045
Epoch 405/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.5045e-05 - mae: 0.0024 - val_loss: 4.6136e-05 - val_mae: 0.0045
Epoch 406/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4876e-05 - mae: 0.0024 - val_loss: 4.8551e-05 - val_mae:

Epoch 457/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4508e-05 - mae: 0.0023 - val_loss: 5.2569e-05 - val_mae: 0.0050
Epoch 458/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4560e-05 - mae: 0.0023 - val_loss: 5.5805e-05 - val_mae: 0.0052
Epoch 459/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4602e-05 - mae: 0.0023 - val_loss: 5.3240e-05 - val_mae: 0.0050
Epoch 460/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4441e-05 - mae: 0.0023 - val_loss: 5.2196e-05 - val_mae: 0.0048
Epoch 461/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4711e-05 - mae: 0.0023 - val_loss: 5.4615e-05 - val_mae: 0.0050
Epoch 462/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4555e-05 - mae: 0.0023 - val_loss: 5.0975e-05 - val_mae: 0.0048
Epoch 463/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4637e-05 - mae: 0.0023 - val_loss: 5.4503e-05 - val_mae:

Epoch 514/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4311e-05 - mae: 0.0023 - val_loss: 5.8296e-05 - val_mae: 0.0051
Epoch 515/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4301e-05 - mae: 0.0023 - val_loss: 5.9528e-05 - val_mae: 0.0053
Epoch 516/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4337e-05 - mae: 0.0023 - val_loss: 5.9379e-05 - val_mae: 0.0052
Epoch 517/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4243e-05 - mae: 0.0023 - val_loss: 5.8530e-05 - val_mae: 0.0051
Epoch 518/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4261e-05 - mae: 0.0023 - val_loss: 5.8989e-05 - val_mae: 0.0051
Epoch 519/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4375e-05 - mae: 0.0023 - val_loss: 6.1048e-05 - val_mae: 0.0055
Epoch 520/1000
704/704 [==============================] - 3s 4ms/step - loss: 1.4351e-05 - mae: 0.0023 - val_loss: 6.2467e-05 - val_mae:

In [32]:
model.evaluate(generator_val)

175/175 [==============================] - 0s 1ms/step - loss: 2.8914e-05 - mae: 0.0030


[2.8914357244502753e-05, 0.0029697634745389223]

In [4]:
np.sqrt(2.8914357244502753e-05)

0.005377207197468101

In [33]:
#model.save("cip_model_best")

In [34]:
# Learning Curves:
plot_history(history)

NameError: name 'plot_history' is not defined

In [ ]:
#Baseline MAE
0.0048812746058099115

# Learning Curves viz code:

In [205]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # Loss:MSE
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # Metrics:MAE
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

In [2]:
np.sqrt(2.8914357244502753e-05)

0.005377207197468101